In [2]:
!git clone https://github.com/ITConctructor/function_calling

Cloning into 'ai_tools'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 58 (delta 17), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (58/58), 21.12 KiB | 7.04 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [1]:
%pip install transformers==4.34.1 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 24.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/8

In [1]:
import os
import sys
import shutil
import json
from tqdm.notebook import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, BitsAndBytesConfig

import warnings
warnings.filterwarnings("ignore")

#Training

In [3]:
from function_calling.src.datasets import FunctionCallingDataset

model_name = "Enoch/llama-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

batch_size = 2
train_data_params = {
    "tokenizer":tokenizer,
    "max_input_len":8096,
    "max_output_len":8096,
    "is_train":True,
    "size":0.0005,
}
train_dataset = FunctionCallingDataset(train_data_params)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate_fn)

val_data_params = {
    "tokenizer":tokenizer,
    "max_input_len":8096,
    "max_output_len":8096,
    "is_train":False,
    "size":0.0005,
}
val_dataset = FunctionCallingDataset(val_data_params)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=val_dataset.collate_fn)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


  0%|          | 0/86753 [00:00<?, ?it/s]

  0%|          | 0/10868 [00:00<?, ?it/s]

In [6]:
from function_calling.src.models import PeftLLM
from function_calling.src.trainer import Trainer

trainer_args = {
    "train_dataloader":train_dataloader,
    "val_dataloader":val_dataloader,
    "score":None,
    "train_score_computing_frequency":1,
    "optimizer":torch.optim.AdamW,
    "optimizer_args":{"lr":3e-4},
    "scheduler":None,
    "scheduler_args":None,
    "freezer":None,
    "freezer_args":None,
    "training_controller":None,
    "n_epochs":1,
    "device":"cuda" if torch.cuda.is_available() else "cpu",
    "backup_path":"/content/drive/MyDrive/ml_studcamp/fc_v1.pt",
    "show_outputs":True,
    "show_outputs_every":1,
    "n_outputs":1,
}

trainer = Trainer(trainer_args)

model_args = {
    "model_name":model_name,
    "weights_dtype":torch.float16,
    "load_in_4bit":True
}
model = PeftLLM(model_args)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
trainer.train(model)

Training:   0%|          | 0/22 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


In [ ]:
print("Trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))
print("Total parameters (excluding quantization):", sum(p.numel() for p in model.parameters()))

Trainable parameters: 8388608
Total parameters (excluding quantization): 3508801536


#Classes

In [4]:
import torch
from torch import nn
import peft
from peft import PeftModel, PeftConfig, LoraConfig
from transformers import AutoModelForCausalLM
import torch.nn.functional as F

class PeftLLM(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.model_name = params["model_name"]
        self.weights_dtype = params.get("weights_dtype", torch.float16)
        load_in_4bit = params.get("load_in_4bit", False)
        self.base_model = AutoModelForCausalLM.from_pretrained(self.model_name, device_map='auto',
        low_cpu_mem_usage=True, offload_state_dict=True, torch_dtype=self.weights_dtype, load_in_4bit=load_in_4bit)
        for param in self.base_model.parameters():
            param.requires_grad = False
        self.base_model.gradient_checkpointing_enable()
        self.base_model.enable_input_require_grads()
        self.tokenizer = params.get("tokenizer", AutoTokenizer.from_pretrained(self.model_name))
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

        self.peft_config = LoraConfig(
            r=16,
            target_modules=["q_proj", "v_proj"],
            task_type=peft.TaskType.CAUSAL_LM,
            lora_alpha=32,
            lora_dropout=0.05
        )
        self.peft_model = peft.get_peft_model(self.base_model, self.peft_config)

        self.loss = nn.CrossEntropyLoss()

    def forward(self, batch):
        inp = {"input_ids":batch["input_text"]['input_ids'], "attention_mask":batch["input_text"]['attention_mask']}
        logits = self.peft_model(**inp).loss["logits"].permute(0, 2, 1)

        out = batch["output_text"]['input_ids']
        loss = self.loss(logits, inp["input_ids"])
        return loss
    def predict(self, batch):
        inp = {"input_ids":batch["input_text"]['input_ids'], "attention_mask":batch["input_text"]['attention_mask']}
        with torch.no_grad():
            generated_ids = self.base_model.generate(**inp)
        out = {
            "out_ids":generated_ids
        }
        return out
    def show_outputs(self, batch):
        inp = {"input_ids":batch["input_text"]['input_ids'], "attention_mask":batch["input_text"]['attention_mask']}
        generated_ids = self.base_model.generate(**inp)
        generated_sentences = self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        print("Generated sentence:")
        for s in generated_sentences:
            print(s)
        return

In [5]:
import torch
from torch import nn
import random
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

class Trainer():
  def __init__(self, params):
    torch.manual_seed(42)
    self.train_dataloader = params["train_dataloader"]
    self.val_dataloader = params["val_dataloader"]
    self.score = params["score"]
    self.train_score_computing_frequency = params.get("train_score_computing_frequency", 1)
    self.optimizer = params["optimizer"]
    self.optimizer_args = params["optimizer_args"]
    self.scheduler = params.get("scheduler", None)
    self.scheduler_args = params.get("scheduler_args", None)
    self.freezer = params.get("freezer", None)
    self.freezer_args = params.get("freezer_args", None)
    self.training_controller = params.get("training_controller", None)
    self.training_controller_params = params.get("training_controller_params", None)
    self.show_outputs = params.get("show_outputs", False)
    self.show_outputs_every = params.get("show_outputs_every", 1)
    self.n_outputs = params.get("n_outputs", 2)
    self.n_epochs = params["n_epochs"]
    self.device = params["device"]
    self.backup_path = params.get("backup_path", None)
    self.history = {
        "train_loss":[],
        "train_score":[],
        "val_loss":[],
        "val_score":[]
    }
  def train(self, model, verbose=True):
    model = model.to(self.device)
    optimizer = self.optimizer(model.parameters(), **self.optimizer_args)
    scheduler = None
    if self.scheduler != None and self.scheduler_args != None:
      scheduler = self.scheduler(optimizer, **self.scheduler_args)
    freezer = None
    if self.freezer != None and self.freezer_args != None:
      freezer = self.freezer(model, **self.freezer_args)

    losses = {
      "train":[],
      "val":[]
    }
    scores = {
      "train":[],
      "val":[]
    }

    if self.training_controller != None:
      self.training_controller_params["model"] = model
      self.training_controller_params["optimizer"] = optimizer
      self.training_controller_params["scheduler"] = scheduler
      self.training_controller_params["losses"] = losses
      self.training_controller_params["scores"] = scores
      controller = self.training_controller(self.training_controller_params)

    for epoch in range(self.n_epochs):
      #Training
      model.train()
      if freezer != None:
        freezer.step()
      train_loss = 0
      train_score = 0
      steps = 0
      train_pbar = tqdm(self.train_dataloader, desc="Training")

      local_losses = []
      local_scores = []
      losses["train"].append(local_losses)
      scores["train"].append(local_scores)
      for batch in train_pbar:
        optimizer.zero_grad()
        train_batch = self.device_dict(batch["train"])
        loss = model(train_batch)
        loss.backward()
        local_losses.append(loss.item())

        val_batch = self.device_dict(batch["val"])
        labels = self.device_dict(batch["labels"])

        if self.score != None:
          preds = model.predict(val_batch)
          if steps % self.train_score_computing_frequency == 0:
            score = self.score(preds, labels)
            local_scores.append(score)

        if self.training_controller != None:
          controller.batch_update()
        optimizer.step()

        display_loss = round(loss.item(), 5)
        train_pbar.set_description("Training. Loss: %s" % display_loss)
        steps += 1



      train_loss = sum(local_losses)/len(local_losses)
      if len(local_scores) > 0:
        train_score = sum(local_scores)/len(local_scores)

      if scheduler != None:
        scheduler.step()

      #Validation
      model.eval()
      val_loss = 0
      val_score = 0
      local_losses = []
      local_scores = []
      losses["val"].append(local_losses)
      scores["val"].append(local_scores)
      for batch in tqdm(self.val_dataloader, desc="Validation"):
        with torch.no_grad():
          train_batch = self.device_dict(batch["train"])

          optimizer.zero_grad()
          loss = model(train_batch)
          local_losses.append(loss.item())
          val_loss += loss.item()/len(self.val_dataloader)

          val_batch = train_batch = self.device_dict(batch["val"])
          labels = train_batch = self.device_dict(batch["labels"])

          if self.score != None:
            preds = model.predict(val_batch)
            score = self.score(preds, labels)
            local_scores.append(score)
            val_score += score/(len(self.val_dataloader))

      self.history["train_loss"].append(train_loss)
      self.history["train_score"].append(train_score)
      self.history["val_loss"].append(val_loss)
      self.history["val_score"].append(val_score)

      if self.training_controller != None:
          controller.epoch_update()

      if verbose:
        print(f"Epoch {epoch+1}. Train loss: {train_loss}, val loss: {val_loss}, train score: {train_score}, val score: {val_score}")

      if self.show_outputs == True and epoch % self.show_outputs_every == 0:
        self.show_outputs_fn(model, self.val_dataloader)

      if self.backup_path != None:
        torch.save(model, self.backup_path)
      self.save_history()

  def save_history(self):
    plt.ioff()
    fig, ax = plt.subplots()
    ax.plot(range(len(self.history["train_loss"])), self.history["train_loss"], label="Train")
    ax.plot(range(len(self.history["val_loss"])), self.history["val_loss"], label="Val")
    fig.savefig("Losses.pdf")

    fig, ax = plt.subplots()
    ax.plot(range(len(self.history["train_score"])), self.history["train_score"], label="Train")
    ax.plot(range(len(self.history["val_score"])), self.history["val_score"], label="Val")
    fig.savefig("Scores.pdf")

  def plot_lr_schedule(self):
    steps = [self.optimizer_args["lr"]]
    if self.scheduler == None or self.scheduler_args == None:
      steps = [self.optimizer_args["lr"] for i in range(self.n_epochs)]
    else:
      dummyModel = nn.Linear(10, 10)
      dummyOptimizer = self.optimizer(dummyModel.parameters(), **self.optimizer_args)
      scheduler = scheduler = self.scheduler(dummyOptimizer, **self.scheduler_args)
      for i in range(1, self.n_epochs):
        dummyOptimizer.step()
        scheduler.step()
        lr = scheduler.get_last_lr()[0]
        steps.append(lr)
    fig, ax = plt.subplots()
    ax.plot(range(len(steps)), steps)
    plt.show()
  def device_dict(self, dictionary):
    for k in dictionary.keys():
      if type(dictionary[k]) == type({}):
        dictionary[k] = self.device_dict(dictionary[k])
      else:
        dictionary[k] = dictionary[k].to(self.device)
    return dictionary
  def show_outputs_fn(self, model, dataloader):
    dataset = dataloader.dataset
    collate_fn = dataloader.collate_fn
    indexes = random.sample(range(len(dataset)), self.n_outputs)
    inputs = [dataset[i] for i in indexes]
    processed = collate_fn(inputs)
    batch = self.device_dict(processed)
    print("Output samples:\n")
    dataset.show_samples(indexes)
    model.show_outputs(batch["val"])


In [2]:
from datasets import load_dataset

class FunctionCallingDataset(Dataset):
    def __init__(self, params):
        self.max_input_len = params["max_input_len"]
        self.max_output_len = params["max_output_len"]
        self.tokenizer = params["tokenizer"]
        self.train = params.get("is_train", True)
        self.size = params.get("size",1)

        self.SYSTEM_PROMPT = (
            "You are a helpful assistant with function-calling supported. You are provided with function signatures within <TOOLS></TOOLS> XML tags. "
            "You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. "
            "Here are the available tools:\n"
            "<TOOLS>\n"
            "{tools}\n"
            "</TOOLS>\n\n"
            "For each function call, return a JSON object with the function name and arguments within <TOOL_CALL></TOOL_CALL> XML tags as follows:\n"
            "<TOOL_CALL>\n"
            "{{\"name\": <function-name>, \"arguments\": <args-dict>}}\n"
            "</TOOL_CALL>"
            "You will get function call result within <TOOL_RESPONSE></TOOL_RESPONSE> XML tags. "
            "Answer user query based on the result."
        )

        self.S_B, self.S_E = "<s>", "</s>"
        self.INST_B, self.INST_E = "[INST] ", " [/INST] "
        self.SYS_B, self.SYS_E = "<<SYS>>\n", "\n<</SYS>>\n\n"
        self.TOOL_CALL_B, self.TOOL_CALL_E = "<TOOL_CALL>\n", "\n</TOOL_CALL>\n\n"
        self.TOOL_RESPONSE_B, self.TOOL_RESPONSE_E = "<TOOL_RESPONSE>\n", "\n</TOOL_RESPONSE>\n\n"

        hf_dataset = load_dataset("korotkov/glaive-function-calling-v2-parsed")

        # Convert dataset to another format
        dataset_key = "train" if self.train else "test"
        self.dataset = []
        self.prompts = []
        self.answers = []
        for row in tqdm(hf_dataset[dataset_key]):
            messages = json.loads(row["messages"])
            functions = json.loads(row["functions"])
            all_text, prompt, answer = self.llama_convert(messages, functions)
            self.dataset.append(all_text)
            self.prompts.append(prompt)
            self.answers.append(answer)
        self.dataset = self.dataset[:int(len(self.dataset)*self.size)]
        self.prompts = self.prompts[:int(len(self.prompts)*self.size)]
        self.answers = self.answers[:int(len(self.answers)*self.size)]

    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, ind):
        text = self.dataset[ind]
        val_text = self.prompts[ind]
        out_text = self.answers[ind]
        item = {
            "train":{
                "input_text":text,
                "output_text":text
            },
            "val":{
                "input_text":val_text
            },
            "labels":{
                "output_text":out_text
            }
        }
        return item
    def show_samples(self, indexes):
        texts = [self.prompts[ind] for ind in indexes]
        print("Prompts:")
        for t in texts:
          print(t)
        return texts
    def llama_convert(self, messages, functions):
      tools = ",\n".join([json.dumps(function, indent=4) for function in functions])
      messages[0]["content"] = self.SYSTEM_PROMPT.format(tools=tools)

      messages_string = [self.S_B, self.INST_B]
      prompts_string = [self.S_B, self.INST_B]
      answers_string = []
      for message in messages:
          if message["role"] == "system":
              messages_string.append(self.SYS_B)
              messages_string.append(message["content"])
              messages_string.append(self.SYS_E)

              prompts_string.append(self.SYS_B)
              prompts_string.append(message["content"])
              prompts_string.append(self.SYS_E)
          elif message["role"] == "user":
              messages_string.append(message["content"])
              messages_string.append(self.INST_E)

              prompts_string.append(message["content"])
              prompts_string.append(self.INST_E)
          elif message["role"] == "assistant":
              messages_string.append(message["content"])
              messages_string.append(self.S_E)

              answers_string.append(message["content"])
              answers_string.append(self.S_E)
          elif message["role"] == "function_call":
              messages_string.append(self.TOOL_CALL_B)
              messages_string.append(message["content"])
              messages_string.append(self.TOOL_CALL_E)

              answers_string.append(self.TOOL_CALL_B)
              answers_string.append(message["content"])
              answers_string.append(self.TOOL_CALL_E)
          elif message["role"] == "function_response":
              messages_string.append(self.TOOL_RESPONSE_B)
              messages_string.append(message["content"])
              messages_string.append(self.TOOL_RESPONSE_E)

              answers_string.append(self.TOOL_RESPONSE_B)
              answers_string.append(message["content"])
              answers_string.append(self.TOOL_RESPONSE_E)

      all_text = "".join(messages_string)
      spl = all_text.rsplit(self.INST_E, 1)
      prompt = spl[0] + self.INST_E
      answer = spl[1]
      return all_text, prompt, answer
    def collate_fn(self, batch):
        texts = ["input_text", "output_text"]
        lens = [self.max_input_len, self.max_output_len]

        new_batch = {
            "train":{
                "input_text":None,
                "output_text":None
            },
            "val":{
                "input_text":None
            },
            "labels":{
                "output_text":None
            }
        }

        for t, l in zip(texts, lens):
            max_batch_len = 0
            texts = [b["train"][t] for b in batch]
            for text in texts:
                tokenized = self.tokenizer([text], return_tensors="pt", padding=True, truncation=True)
                length = tokenized["input_ids"].shape[1]
                if length > max_batch_len:
                    max_batch_len = length
            max_batch_len = min(max_batch_len, l)
            for k in new_batch.keys():
                tokenized = self.tokenizer(texts, return_tensors="pt", max_length=max_batch_len, padding="max_length", truncation=True)
                if t == "input_text":
                  tokenized["input_ids"] = tokenized["input_ids"][:,:-1]
                  tokenized["attention_mask"] = tokenized["attention_mask"][:,:-1]
                else:
                  tokenized["input_ids"] = tokenized["input_ids"][:,1:]
                  tokenized["attention_mask"] = tokenized["attention_mask"][:,1:]
                if t in new_batch[k]:
                    if k != "train":
                      true_texts = [b[k][t] for b in batch]
                      tokenized = self.tokenizer(true_texts, padding=True, return_tensors="pt")
                    new_batch[k][t] = tokenized

        return new_batch